In [12]:
from conformal_sparsemax.classifier import CNN,get_data, evaluate
from sklearn.metrics import f1_score
import torch
from entmax.losses import SparsemaxLoss, Entmax15Loss

In [13]:
loss = 'sparsemax' #sparsemax, softmax or entmax15
dataset='CIFAR100'

In [20]:
model = CNN(loss)
_,_, test_dataloader, cal_dataloader = get_data(0.2,16)
model.load_state_dict(torch.load(f'models/{dataset}_{loss}.pth'))
if loss == 'sparsemax':
    criterion = SparsemaxLoss()
elif loss == 'entmax15':
    criterion = Entmax15Loss()
elif loss == 'softmax':
    criterion = torch.nn.NLLLoss()

Files already downloaded and verified
Files already downloaded and verified


In [11]:
pred_proba, pred_labels, true_labels, test_loss = evaluate(
                                                    model,
                                                    test_dataloader,
                                                    criterion)

f1 = f1_score(true_labels, pred_labels, average='weighted')

print(f'Test loss: {test_loss:.3f}')
print(f'Test f1: {f1:.3f}')

Test loss: 0.447
Test f1: 0.257


In [47]:
print('type:', type(pred_proba))
print('pred_proba shape:',pred_proba.shape)
print('pred_labels shape:',pred_labels.shape)
print('true_labels shape:',true_labels.shape)

type: <class 'numpy.ndarray'>
pred_proba shape: (10000, 100)
pred_labels shape: (10000,)
true_labels shape: (10000,)


In [43]:
pred_proba[pred_proba==0].size / pred_proba.size

0.920259